<a href="https://colab.research.google.com/github/n-bzy/iannwtf/blob/main/homework08_david_unfinished_loss_does_not_change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
import tqdm
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import layers, losses
import datetime

train_ds, test_ds = tfds.load('mnist', split =['train', 'test'], as_supervised = True)

def preprocessing(mnist, batch_size):

    #get rid of the targets
    mnist = mnist.map(lambda img, target: (img))

    #change datatype from unit-8 to float
    mnist = mnist.map(lambda img: (tf.cast(img, tf.float32)))

    #normalize image values
    mnist = mnist.map(lambda img: ((img/128.)-1))

    #shape: (28,28,1) values in range 0 to 1

    #add noise to input images, old images as targets
        #random noise tensor
    noise = tf.random.uniform(shape=(28,28,1), minval = 0, maxval = 1, dtype=tf.float32)
        ##add noise tensor to input img
    mnist = mnist.map(lambda img: ((tf.math.add(img, noise), img)))
    mnist = mnist.map(lambda img1, img2: (tf.clip_by_value(img1, clip_value_max=0, clip_value_min=1), img2))
        #values in input images are now 0 to 2
        #so divide by to to have range of 0 to 1 again
    #mnist = mnist.map(lambda noise_img, img : ((noise_img/2), img))

    #could have used tf.clip.by_value(,min=,max=) instead

    #add third dimension for autoencoder
    mnist = mnist.map(lambda noise_img, img: (tf.expand_dims(noise_img, -1), tf.expand_dims(img, -1)))

    #now we should have tuple (noisy_img, img) which each have the shape (28,28,1,1) and range from 0 to 1

    #shuffle, batch, prefetch
    mnist = mnist.shuffle(2000)
    mnist = mnist.batch(batch_size)
    mnist = mnist.prefetch(tf.data.AUTOTUNE)

    return mnist

train_ds = preprocessing(train_ds, 256)
test_ds = preprocessing(test_ds, 256)

#check shape
for noise_img, img in train_ds.take(1):
    print(noise_img.shape, img.shape)

#We have tuple (noisy_img, img) which each have the shape (batch_size, 28,28,1,1) and range from 0 to 1 in the dimension with the first 1 (i hope)
#!! maybe i actually dont need to non-noisy img as targets

#Convolutional Autoencoder to Denoise

class Denoise(tf.keras.Model):
    def __init__(self):
        super(Denoise, self).__init__()

        #Encoder layers:
        self.layer1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding="same",strides=2, activation="relu")
            #Image size 14x14
        self.layer2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding="same",strides=2, activation="relu")
            #Image size 7x7
        #Flatten Image
        self.flatten = tf.keras.layers.Flatten()
            #Vector size (batch_size, 49)

        #Create Embedding of size 10 with dense layer (this is not a dense layer is it??)
        #self.embedding = tf.keras.layers.Embedding(50, 10)

        #aus silvies code
        self.bottleneck_in = tf.keras.layers.Dense(7*7*128, activation='relu')
        self.embedding = tf.keras.layers.Dense(10)
        #bis hier

        #Decoder layers:
        #aus silvies code:
        self.bottleneck_out = tf.keras.layers.Dense(7*7*128, activation='relu')
        self.reshape = tf.keras.layers.Reshape((7,7,128))
        #bis hier
            #Shape now (batch_size, 7, 7, 128)
        self.re_layer1 = tf.keras.layers.Conv2DTranspose(filters=128, kernel_size=3, padding="same", strides=2, activation="relu")
            #Img size 14x14
        self.re_layer2 = tf.keras.layers.Conv2DTranspose(filters=64, kernel_size=3, padding="same", strides=2, activation="relu")
            #Img size 28x28
        #Output layer
        self.output_layer = tf.keras.layers.Conv2D(filters=1, kernel_size=3, padding="same", activation="sigmoid")

    @tf.function
    def encoder(self, input):
        x = self.layer1(input)
        x = self.layer2(x)
        x = self.flatten(x)
        x = self.bottleneck_in(x)
        x = self.embedding(x) 

        return x

    @tf.function
    def decoder(self, encoder_output):
        x = self.bottleneck_out(encoder_output)
        x = self.reshape(x)
        x = self.re_layer1(x)
        x = self.re_layer2(x)
        x = self.output_layer(x)

        return x

    @tf.function
    def __call__(self, input, training=False):
        encoded = self.encoder(input)
        decoded = self.decoder(encoded)

        return decoded

    @tf.function
    def train_step(self, data):
        x, t = data
        with tf.GradientTape() as tape:
            output = self(x, training=True)
            loss = self.compiled_loss(t, output, regularization_losses=self.losses)
        gradients = tape.gradient(loss, self.trainable_variables)

        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        self.metrics[0].update_state(loss)
        
        return {m.name : m.result() for m in self.metrics}

    @tf.function
    def test_step(self, data):
        x, t = data
        output = self(x, training=True)
        loss = self.compiled_loss(t, output, regularization_losses=self.losses)
        
        self.metrics[0].update_state(loss)

        return {m.name : m.result() for m in self.metrics}

autoencoder = Denoise()    
autoencoder.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.5), loss=losses.MeanSquaredError())

autoencoder.fit(train_ds,
                validation_data = test_ds,
                epochs = 10
                )

    




(256, 28, 28, 1, 1) (256, 28, 28, 1, 1)
Epoch 1/10
235/235 [==============================] - 31s 125ms/step - loss: 0.9282 - val_loss: 0.9243
Epoch 2/10
235/235 [==============================] - 28s 120ms/step - loss: 0.9239 - val_loss: 0.9243
Epoch 3/10
235/235 [==============================] - 29s 122ms/step - loss: 0.9239 - val_loss: 0.9243
Epoch 4/10
235/235 [==============================] - 29s 120ms/step - loss: 0.9239 - val_loss: 0.9243
Epoch 5/10
235/235 [==============================] - 29s 121ms/step - loss: 0.9239 - val_loss: 0.9243
Epoch 6/10
235/235 [==============================] - 28s 119ms/step - loss: 0.9239 - val_loss: 0.9243
Epoch 7/10
235/235 [==============================] - 28s 120ms/step - loss: 0.9239 - val_loss: 0.9243
Epoch 8/10
235/235 [==============================] - 29s 122ms/step - loss: 0.9239 - val_loss: 0.9243
Epoch 9/10
235/235 [==============================] - 29s 120ms/step - loss: 0.9239 - val_loss: 0.9243
Epoch 10/10
235/235 [============